# Рекомендация тарифов <a id='intro'></a>

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «Определение выгодного тарифа для телеком компании»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Необходимо построить модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75.

## Оглавление

### [1.Откроем и изучим файл](#glava1)
### [2.Разобъем данные на выборки](#glava2)
### [3.Исследуем модели](#glava3)
#### [3.1 Решающее дерево](#model1)
#### [3.2 Случайный лес](#model1)
#### [3.3 Логистическая регрессия](#model3)
### [4.Проверим модель на тестовой выборке](#glava4)
### [Итоговый вывод](#glava6)

## Откроем и изучим файл <a id='glava1'></a>

**Импортируем библиотеки**

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

#import warnings
#warnings.simplefilter("ignore")

from tqdm.notebook import trange, tqdm
from time import sleep

**Считаем данные из csv-файла в датафрейм и сохраним в переменную df. Путь к файлу:**
`/datasets/users_behavior.csv`

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Выводим первые 10 строчек датафрейма df на экран

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- **сalls** — количество звонков,
- **minutes** — суммарная длительность звонков в минутах,
- **messages** — количество sms-сообщений,
- **mb_used** — израсходованный интернет-трафик в Мб,
- **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» —1, «Смарт» — 0).

**Выведим основную информацию о датафрейме.**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Кол-во значений одинаково. Следовательно пропущенные значения нет. Также значения в столбцах **сalls** и **messages** не соовтесвуют своим типам данных. Предлагается перевести в целочисленный тип `int64`.

**Перевод в тип данных `int64`**

In [5]:
df['calls'] = df['calls'].astype('int64')

In [6]:
df['messages'] = df['messages'].astype('int64')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [8]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0
5,58,344.56,21,15823.37,0
6,57,431.64,20,3738.90,1
7,15,132.40,6,21911.60,0
8,7,43.39,3,2538.67,1
9,90,665.41,38,17358.61,0


**Вывод по главе.**
Был загружен файл и изучен. Аномалий в нем не замечено и ошибок тоже. Была сделана корреткировка данных по столбцам сalls и messages. Данные этих столбцов были переведены в целочисленный тип int64

[Начало тетрадки](#intro).

## Разобъем данные на выборки <a id='glava2'></a>

Задание: Разделение исходных данных на обучающую, валидационную и тестовую выборки.

Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.

Значит под обучающие данные мы выделим 60%.
Под валидационную часть 20% и под тестовую часть тоже 20%

Выделим для начала обучающую часть и "остальную" часть в соотношении 60% и 40%

In [9]:
# df_train - обучающаяся выборка
# df_other - оставшаяся часть, которую разобьем на тестовую и валидационную

df_train, df_other = train_test_split(df, test_size=0.4, random_state=12345)

Разобьем оставшуюся часть на валидационную и тестовую в соотношении 50% на 50%

In [10]:
# df_valid - валидационная выборка
# df_test - тестовая выборка

df_valid, df_test = train_test_split(df_other, test_size=0.5, random_state=12345)

Выделим признаки и целевые признаки у обучающей части

In [11]:
features = df_train.drop(['is_ultra'], axis=1) 
target = df_train['is_ultra']

Выделим признаки и целевые признаки у валидационной части

In [12]:
valid_features = df_valid.drop(['is_ultra'], axis=1) 
valid_target = df_valid['is_ultra']

Выделим признаки и целевые признаки у тестовой части

In [13]:
test_features = df_test.drop(['is_ultra'], axis=1) 
test_target = df_test['is_ultra']

**Вывод по главе.**
Было проведено разделение исходных данных на обучающую(тренировочную), валидационную и тестовую выборки в пропорции 60%:20%:20%

[Начало тетрадки](#intro).

## Исследуем модели <a id='glava3'></a>

В данной работе рассмотрим 3 модели:

* Решающее дерево
* Случайный лес
* Логистическая регрессия

Меняя гиперпараметры каждой модели выберим наилучшую. 

**Решающее дерево** <a id='model1'></a>

Все гиперпараметры DecisionTreeClassifier: 
* criterion='gini' 
* splitter='best'
* max_depth=None
* min_samples_split=2
* min_samples_leaf=1
* min_weight_fraction_leaf=0.0
* max_features=None
* random_state=None
* max_leaf_nodes=None
* min_impurity_decrease=0.0 
* class_weight=None
* ccp_alpha=0.0

Выделим 3 гиперпараметра и попробуем улучшить модель.
Для исследования выберем 
- max_depth в диапазоне от 1 до 10
- criterion с значениями “gini”, “entropy”, “log_loss”
- min_samples_split (от англ. «минимальное количество примеров для разделения»). Этот гиперпараметр запрещает создавать узлы, в которые попадает слишком мало объектов обучающей выборки. Должно быть не менее 2.
Остальные гиперпараметры оставим по умолчанию.

In [14]:
import sklearn
print(sklearn.__version__)    

1.1.1


In [15]:
##%%time
best_model1 = None 
best_result = 0
best_depth1 = 0
best_criterion = 0
best_samples = 0

for i in tqdm(["gini", "entropy"]):
    for depth in range(1, 10):
        for samples in range(2, 40):
            model1 = DecisionTreeClassifier(criterion=i, 
                                            max_depth=depth, 
                                            random_state=12345, 
                                            min_samples_split=samples) 
            # обучение модели
            model1.fit(features, target) 
            
            # получение предсказания модели 
            predictions = model1.predict(valid_features) 
            
            # подсчет качества модели
            result = accuracy_score(valid_target, predictions)
             
            if result > best_result:
                best_model1 = model1 
                best_result = result
                best_depth1 = depth
                best_criterion = i
                best_samples = samples
            sleep(0.01)
                
print('Процесс завершен')  
print('')
print("Accuracy лучшей модели:", best_result)
print("Лучшая max_depth:", best_depth1)
print("Лучшая criterion:", best_criterion)
print("Лучшая min_samples_split:", best_samples)

  0%|          | 0/2 [00:00<?, ?it/s]

Процесс завершен

Accuracy лучшей модели: 0.7900466562986003
Лучшая max_depth: 9
Лучшая criterion: entropy
Лучшая min_samples_split: 21


При данном max_depth нет смысла увеличивать min_samples_split относительно лучшей, так как значение Accuracy не меняется. Уменьшать тоже не смысла так как сразу ухудшится Accuracy и изменится глубина дерева в меньшую сторону и criterion изменится на gini

**Случайный лес** <a id='model2'></a>

Все гиперпараметры RandomForestClassifier: 

* n_estimators=100
* criterion='gini'
* max_depth=None
* min_samples_split=2
* min_samples_leaf=1
* min_weight_fraction_leaf=0.0
* max_features='sqrt'
* max_leaf_nodes=None
* min_impurity_decrease=0.0
* bootstrap=True
* oob_score=False
* n_jobs=None
* random_state=None
* verbose=0
* warm_start=False
* class_weight=None
* ccp_alpha=0.0
* max_samples=None)

Выделим 2 гиперпараметра и попробуем улучшить модель.
Для исследования выберем 
- n_estimators в диапазоне от 1 до 100 (по умолчанию 100)
- max_depth 
Остальные гиперпараметры оставим по умолчанию.


In [16]:
%%time
best_model2 = None 
best_result = 0
best_depth2 = 0
best_est = 0

for est in tqdm(range(1, 100)):
    for depth in range(1, 9):
        model2 = RandomForestClassifier(random_state=12345, 
                                        criterion='gini', 
                                        n_estimators=est, 
                                        max_depth = depth) 
        # обучение модели
        model2.fit(features, target) 
        # получение предсказания модели 
        predictions = model2.predict(valid_features) 
        # подсчет качества модели
        result = accuracy_score(valid_target, predictions)

        if result > best_result:
            best_model2 = model2 
            best_result = result
            best_depth2 = depth
            best_est = est
        sleep(0.01)    
print('Процесс завершен')  
print('')
print("Accuracy лучшей модели:", best_result)
print("Лучшая max_depth:", best_depth2)
print("Лучшая n_estimators:", best_est)

  0%|          | 0/99 [00:00<?, ?it/s]

Процесс завершен

Accuracy лучшей модели: 0.8087091757387247
Лучшая max_depth: 8
Лучшая n_estimators: 40
CPU times: user 1min 11s, sys: 665 ms, total: 1min 12s
Wall time: 1min 21s


При рассчитанных параметрах при увеличении n_estimators и max_depth Accuracy не меняется, так как найдено максимальное знаечние для лучшей модели. А увеличение также приводит к увеличениб времени расчетов.

**Логистическая регрессия** <a id='model3'></a>

Все гиперпараметры LogisticRegression:
* penalty='l2' 
* dual=False
* tol=0.0001
* C=1.0
* fit_intercept=True
* intercept_scaling=1
* class_weight=None
* random_state=None
* solver='lbfgs'
* max_iter=100
* multi_class='auto'
* verbose=0
* warm_start=False
* n_jobs=None
* l1_ratio=None)

Выделим 1 гиперпараметр и попробуем улучшить модель.
Для исследования выберем 
- max_iter в диапазоне от 1 до 500 (задаётся максимальное количество итераций обучения)

Остальные гиперпараметры оставим по умолчанию.


In [17]:
%%time
best_model3 = None 
best_result = 0
best_iter = 0


for iteration in range(1, 500):
    model3 = LogisticRegression(random_state=12345, 
                                solver='lbfgs', 
                                max_iter=iteration)
    
    # обучение модели
    model3.fit(features, target) 
    
    # получение предсказания модели 
    predictions = model3.predict(valid_features) 
    
    # подсчет качества модели
    result = accuracy_score(valid_target, predictions)
    
    if result > best_result:
        best_model3 = model3 
        best_result = result
        best_iter = iteration
                 
print("Accuracy лучшей модели:", best_result)
print("Лучшая max_iter:", best_iter)

/Users/manyak_pro/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/manyak_pro/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

/Users/manyak_pro/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/manyak_pro/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

Accuracy лучшей модели: 0.7107309486780715
Лучшая max_iter: 39
CPU times: user 42.3 s, sys: 602 ms, total: 42.9 s
Wall time: 5.38 s


Наилучшее значение получено при max_iter=39. Результат нам не подходит так как меньше 0.75.

**Вывод по главе.**
Было проведено исследование как гиперпараметры влияют на результат в обучаемой модели. Для каждой модели рассмотрели разные гиперпараметры и в разном кол-ве. При нашем исследовании лучшей моделью оказалась RandomForestClassifier. Однако модель полученная с помощью DecisionTreeClassifier тоже подходит, так больше 0.75 и близка к значению полученного у модели случайный лес.

[Начало тетрадки](#intro).

## Проверим модель на тестовой выборке <a id='glava4'></a>

В предыдущей главе была выбрана лучшая модель. Ею оказалась Случайный лес. Проверим модель на тестовой выборке с лучшими гиперпараметрами.

In [18]:
%%time
model = RandomForestClassifier(random_state=12345, 
                                        criterion='gini', 
                                        n_estimators=best_est, 
                                        max_depth = best_depth2) 
# обучение модели
model.fit(features,target) 
# получение предсказания модели 
predictions1 = model.predict(test_features) 
# подсчет качества модели
result1 = accuracy_score(test_target, predictions1)
print("Accuracy наилучшей модели на тестовой выборке:", result1)

Accuracy наилучшей модели на тестовой выборке: 0.7962674961119751
CPU times: user 758 ms, sys: 15.4 ms, total: 774 ms
Wall time: 96.9 ms


In [19]:
# второй способ
#best_model2.fit(test_features,test_target) # обученная модель на тренировочной выборке 
#predictions_valid = best_model2.predict(valid_features) # получение предсказания модели на валидационной выборке
#result = accuracy_score(valid_target, predictions_valid) # посчет качества модели
#print("Accuracy наилучшей модели на тестовой выборке:", result)

**Вывод по главе.** Значение Accuracy на тестовой выборке больше 0.75. Следовательно была выбрана удачная модель и подобраны необходимые гиперпараметры у нее. 
Значения на Accuracy на тестовой выборке 0.7962674961119751, а на валидационной выборке 0.8087091757387247. Отклонения незначительные (в пределах 0,012) . Возможно модель слегка переучена.

[Начало тетрадки](#intro).

## Итоговый вывод <a id='glava6'></a>

**Общий вывод**
В вашем распоряжении были представлены данные о поведении клиентов, которые уже перешли на тарифы «Ультра» и «Смарт» (из проекта курса «Статистический анализ данных»). 

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не проводилась, так как она была сделана ранее в проекте. Необходимо построить модель с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 0.75. А также проверить accuracy на тестовой выборке.


* **Шаг 1**

Загрузка данных. Импортирование библиотек. Аномалий в данных не замечено и ошибок тоже. Была сделана корреткировка данных по столбцам сalls и messages. Данные этих столбцов были переведены в целочисленный тип int64

* **Шаг 2**
Разделение исходных данных происходило на обучающую, валидационную и тестовую выборки.
Исходные данные разбили в соотношении 3:1:1. Т.е.под обучающую выборку было выделено 60%, под валидационную и тестовую по 20%

* **Шаг 3**

В данной работе были рассмотренны 3 модели:

* Решающее дерево (рассмотрено 3 гиперпараметра)

  - Accuracy лучшей модели: 0.7900466562986003 
  - Лучшая max_depth: 9 
  - Лучшая criterion: entropy
  - Лучшая min_samples_split: 21

* Случайный лес (рассмотрено 2 гиперпараметра)

  - Accuracy лучшей модели: 0.8087091757387247
  - Лучшая max_depth: 8
  - Лучшая n_estimators: 40

* Логистическая регрессия (рассмотрен 1 гиперпараметр)
  - Accuracy лучшей модели: 0.7107309486780715
  - Лучшая max_iter: 39

Было проведено исследование как гиперпараметры влияют на результат в обучаемой модели. Для каждой модели рассмотрели разные гиперпараметры и в разном кол-ве. При нашем исследовании лучшей моделью оказалась RandomForestClassifier. Однако модель полученная с помощью DecisionTreeClassifier тоже подходит, так больше 0.75 и близка к значению полученного у модели случайный лес.

* **Шаг 4**

Значение Accuracy на тестовой выборке больше 0.75. Следовательно была выбрана удачная модель и подобраны необходимые гиперпараметры у нее. 
Значения на Accuracy на тестовой выборке 0.7962674961119751, а на валидационной выборке 0.8087091757387247. Отклонения незначительные (в пределах 0,012) . Возможно модель слегка переучена.

[Начало тетрадки](#intro).